# Spatial Join of Points to Lines

This notebook demonstrates how to perform a spatial join between point and line geometries. It's a common task in GIS when you need to associate assets (like hydrants or meters) with the pipes they are connected to.

The process involves:
1. Loading line data (e.g., water mains).
2. Loading point data (e.g., hydrants, valves).
3. Performing a spatial join to find which lines each point intersects with.
4. For points that don't directly intersect a line, performing a buffered spatial join to find the nearest line within a certain tolerance.

## 1. Imports

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
import os

## 2. Configuration

Define the layers you want to process and the tolerance for buffered joins.

In [ ]:
POINT_LAYERS = [
    "wNetworkMeter",
    "wChamber",
    "wNetworkOptValve",
    "wOperationalSite",
    "wPressureContValve",
    "wPressureFitting",
    "wHydrant",
]

# Point layers which require a spatial join with tolerance, tolerance in meters
# Needs to be small to limit incorrect matches, but large enough to match assets
TOLERANCES = {"wOperationalSite": 1, "wChamber": 0.5}

## 3. Data Loading

Here, we'll load the water mains data. You can either load it from the provided sample file or connect to the PostGIS database if you have it running.

In [ ]:
# Option 1: Load from the sample GeoPackage file
# Make sure the path is correct relative to the notebook's location.
shape_file = "../../data/sample_data.gpkg" # Using GeoPackage for better compatibility

if not os.path.exists(shape_file):
    raise FileNotFoundError(f"Sample data not found at {shape_file}. Please ensure the file exists.")

gdf_mains = gpd.read_file(shape_file, layer="wTrunkMain")
gdf_mains = gdf_mains[["GISID", "geometry"]]
gdf_mains.set_index("GISID", inplace=True)
gdf_mains.dropna(subset=["geometry"], inplace=True)

print(f"Loaded {len(gdf_mains)} mains from {shape_file}")

## 4. Helper Functions

In [ ]:
def join_points_on_lines(
    points: gpd.GeoDataFrame, lines: gpd.GeoDataFrame, tol: float = None
) -> gpd.GeoDataFrame:
    """Spatial join of points to lines."""

    p = points[["geometry"]]
    l = lines[["geometry"]]

    if tol:
        # Create circles with radius tol around points
        p["geometry"] = gpd.GeoSeries(p["geometry"]).buffer(tol)
    
    # Spatial join of points (or circles if tol) to lines
    df = gpd.sjoin(p, l, how="inner")
    df.reset_index(inplace=True)
    df.rename(
        columns={
            "index": "Point",
            "index_left": "Point",
            "GISID": "Point",
            "index_right": "Line",
        },
        inplace=True,
    )
    df = df[["Point", "Line"]]

    if tol:
        # Removal of points which match to multiple lines when using tolerance
        df["count"] = df.groupby("Point")["Point"].transform("count")
        df = df[df["count"] == 1].drop(columns="count")

    return df

## 5. Main Processing Loop

This loop iterates through each point layer, performs the spatial join, and collects the results.

In [ ]:
all_mappings = []

for layer in POINT_LAYERS:
    print(f"Processing layer: {layer}")
    try:
        gdf_points = gpd.read_file(shape_file, layer=layer)
        gdf_points["layer"] = layer
        gdf_points.set_index("GISID", inplace=True)
        gdf_points.dropna(subset=["geometry"], inplace=True)
        print(f"...read in {len(gdf_points)} records")

        # Perform direct spatial join (no tolerance)
        map_point_line = join_points_on_lines(gdf_points, gdf_mains, None)

        # Perform buffered join for points that didn't match
        if TOLERANCES.get(layer):
            print("  Finding points within tolerance...")
            # Filter to points that were not already joined
            unjoined_points = gdf_points[~gdf_points.index.isin(map_point_line["Point"])]
            
            if not unjoined_points.empty:
                gdf_tol = join_points_on_lines(
                    unjoined_points, gdf_mains, tol=TOLERANCES.get(layer)
                )
                print(f"  ...found {len(gdf_tol)} additional matches with tolerance.")
                map_point_line = pd.concat([map_point_line, gdf_tol], ignore_index=True)

        print(f"...found {len(map_point_line)} total intersections for {layer}")
        map_point_line["layer"] = layer
        all_mappings.append(map_point_line)

    except Exception as e:
        print(f"Could not process layer {layer}. It might not exist in the shapefile. Error: {e}")

# Combine all results into a single DataFrame
if all_mappings:
    final_map = pd.concat(all_mappings, ignore_index=True)
    print("\nProcessing complete.")
    print(f"Total mappings found: {len(final_map)}")
else:
    print("\nProcessing complete. No mappings found.")
    final_map = pd.DataFrame()

## 6. View and Save Results

In [ ]:
if not final_map.empty:
    print(final_map.head())
    
    # Save the results to a CSV file
    output_path = "../../results/point_to_line_mappings.csv"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    final_map.to_csv(output_path, index=False)
    print(f"\nResults saved to {output_path}")